To use this file, you need to load datasets from `sharedata/data_for_compute_similarity/data` into your `data` folder, where there are 5 datasets.

that is, you may use the command `cp -r /shareddata/data_for_compute_similarity/data/ .`

Additionally, you also need to load clip model by the command `cp -r /data/lab/STA303-Assignment02/clip/ .`   

## Data preparation and import packages

In [14]:
import os
import shutil

deal with the mini-imagenet


In [15]:
# # 定义数据集目录
# dataset_dir = "data/mini-imagenet/images/"

# os.remove("data/mini-imagenet/images/.DS_Store")
# # 遍历数据集目录下的所有图片文件
# for filename in os.listdir(dataset_dir):
#     if filename.endswith(".jpg"):
#         # 除去文件名的后四位数字
#         class_id = filename[0:9]
        
#         # 构建目标类别目录
#         target_class_dir = os.path.join(dataset_dir, class_id)
#         os.makedirs(target_class_dir, exist_ok=True)
        
#         # 构建源文件路径和目标文件路径
#         source_path = os.path.join(dataset_dir, filename)
#         target_path = os.path.join(target_class_dir, filename)
        
#         # 移动文件到目标类别目录
#         shutil.move(source_path, target_path)

# print("操作完成。")
# tmp = os.listdir(dataset_dir)
# print(len(tmp))

merge the training set and testing set (`Plant disease` and `standford cars`) to the directory `images`

In [16]:
# def merge_dataset(dataset_dir):
#     # 定义训练集和测试集目录
#     train_dir = os.path.join(dataset_dir, "train")
#     test_dir = os.path.join(dataset_dir, "test")

#     # 获取训练集和测试集中的类别
#     train_classes = set(os.listdir(train_dir))
#     test_classes = set(os.listdir(test_dir))

#     # 检查训练集和测试集中的类别是否一致
#     if train_classes != test_classes:
#         raise ValueError("训练集和测试集中的类别不一致，请检查数据集。")

#     for subset_dir in [train_dir, test_dir]:
#         for class_name in os.listdir(subset_dir):
#             class_dir = os.path.join(subset_dir, class_name)
            
#             # 遍历类别目录下的所有图片文件
#             for filename in os.listdir(class_dir):
#                 if filename.endswith(".jpg"):
#                     # 构建目标类别目录
#                     target_class_dir = os.path.join(dataset_dir, "images", class_name)
#                     os.makedirs(target_class_dir, exist_ok=True)
                    
#                     # 构建源文件路径和目标文件路径
#                     source_path = os.path.join(class_dir, filename)
#                     target_path = os.path.join(target_class_dir, filename)
                    
#                     # 移动文件到目标类别目录
#                     shutil.move(source_path, target_path)


#     shutil.rmtree(train_dir)
#     shutil.rmtree(test_dir)

#     print("操作完成。")

In [17]:
# merge_dataset('data/Plant_disease/')
# merge_dataset('data/stanford_cars/')

In [18]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision.datasets import CIFAR10
from torchvision import datasets
from torchvision import transforms
import torchvision

from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from clip import clip
import pickle

from torchvision.transforms import Compose, Resize, ToTensor, Normalize

import math
from tqdm import tqdm

## load Clip

In [30]:
VISUAL_BACKBONE = 'RN50' # RN50, ViT-B/32, ViT-B/16

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model, transform = clip.load(VISUAL_BACKBONE, device=device)
model.to(device);

In [20]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device)

In [21]:
# ## try one image 
# image_path = "data/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg"

# image_tensor = preprocess_image(image_path)
# vec_features = model.encode_image(image_tensor)
# print(vec_features)
# print(vec_features.shape)

## Define functions

$Sim(D_B,D_N) = exp(-\alpha ~EMD(D_B,D_N))$

where 
$EMD(D_B,D_N) = \frac{\sum_{i \in C_B, ~j \in C_N}{f_{i,j}~d_{i,j}}}{\sum_{i \in C_B, ~j \in C_N}{f_{i,j}}},~~\alpha ~\text{is typically set to} ~ 0.01$

where $d_{i,j} = \Vert p_i-p_j \Vert_2 $ 

In [28]:
# CUB
CUB_path = 'data/CUB_200_2011/images/'

# Stanford Cars
Cars_path = 'data/stanford_cars/images/'

# EuroSAT
EuroSAT_path = 'data/EuroSAT_RGB/'

# Plant disease
Plant_path = 'data/Plant_disease/images'

# mini-imagenet
mini_path = 'data/mini-imagenet/images/'

alpha = 0.01

计算源域的向量

In [31]:
# 分别得到两个数据集的标签集
mini_class = os.listdir(mini_path)
    
with torch.no_grad():
    # 遍历第一个数据集的标签
    mini_vectorlist = [] 
    for i in tqdm(mini_class):   
        ##* 对于 dataset1 的 i类 图像， 提取出平均特征 p_i
        class_i = os.path.join(mini_path, i)
        
        p_i = 0
        
        # 遍历类别下的每张图片
        for image_name in os.listdir(class_i):
            image_path = os.path.join(class_i, image_name)
            image_tensor = preprocess_image(image_path)
            image_tensor = image_tensor.to(device)
            p_i += model.encode_image(image_tensor)
        
        p_i = p_i / len(os.listdir(class_i)) 

        mini_vectorlist.append(p_i)


# 创建一个文件夹叫vectorlist，如果它不存在的话
vectorlist_folder = 'vectorlist_for_similarity'
os.makedirs(vectorlist_folder, exist_ok=True)

# 保存 mini_vectorlist
vectorlist_file_path = os.path.join(vectorlist_folder, 'mini_vectorlist.pkl')
with open(vectorlist_file_path, 'wb') as f:
    pickle.dump(mini_vectorlist, f)

 10%|█         | 10/100 [06:48<1:01:18, 40.87s/it]


KeyboardInterrupt: 

In [ ]:
# 加载 mini_vectorlist
vectorlist_file_path = 'vectorlist/mini_vectorlist.pkl'
with open(vectorlist_file_path, 'rb') as f:
    loaded_mini_vectorlist = pickle.load(f)

源域与数据集之间的similarity


In [ ]:
#源域与数据集之间的similarity
def similarity_withSource(target_path):
    # 分别得到 target 数据集的标签列表
    target_class = os.listdir(target_path)
        
    with torch.no_grad():
        # 遍历第二个数据集的标签
        target_vectorlist = []
        for j in tqdm(target_class):
            ##* 对于 target_path 的 第j类 图像，提取出平均特征 p_j
            class_j = os.path.join(target_path, j)
            
            p_j = 0
            # 遍历类别下的每张图片
            for image_name in os.listdir(class_j):
                image_path = os.path.join(class_j, image_name)
                image_tensor = preprocess_image(image_path)
                image_tensor = image_tensor.to(device)
                p_j += model.encode_image(image_tensor)
            
            p_j = p_j / len(os.listdir(class_j))    
            target_vectorlist.append(p_j)        
            
        # 保存 mini_vectorlist
        
        # 使用 split 方法获取目标字符串
        start_index = target_path.find('data/') + len('data/')
        end_index = target_path.find('/', start_index)
        result_string = target_path[start_index:end_index]
        
        vectorlist_file_path = os.path.join("vectorlist_for_similarity", result_string,'_vectorlist.pkl')
        with open(vectorlist_file_path, 'wb') as f:
            pickle.dump(target_vectorlist, f)
    
        # 创建 权重矩阵
        weight_matrix = np.ones((len(loaded_mini_vectorlist),len(target_vectorlist)))  
        weight_matrix = weight_matrix/np.sum(weight_matrix)
         
        # 计算
        EMD = 0
        for i in range(len(loaded_mini_vectorlist)):
            for j in range(len(target_vectorlist)):
                
                ##* 将两个求二范数  d_ij
                EMD += weight_matrix[i][j] * torch.norm(loaded_mini_vectorlist[i] - target_vectorlist[j], p=2)

    return math.e ** (-1*alpha*EMD)    

任意两个数据集之间的similarity

In [29]:
# 任意两个数据集之间的similarity

# def similarity(dataset1,dataset2):
#     # 分别得到两个数据集的标签列表
#     label_class1 = os.listdir(dataset1)
#     label_class2 = os.listdir(dataset2)
        
#     with torch.no_grad():
#         # 遍历第一个数据集的标签
#         dataset1_vectorlist = [] 
#         for i in tqdm(label_class1):   
#             ##* 对于 dataset1 的 i类 图像， 提取出平均特征 p_i
#             class_i = os.path.join(dataset1, i)
            
#             p_i = 0
            
#             # 遍历类别下的每张图片
#             for image_name in os.listdir(class_i):
#                 image_path = os.path.join(class_i, image_name)
#                 image_tensor = preprocess_image(image_path)
#                 image_tensor = image_tensor.to(device)
#                 p_i += model.encode_image(image_tensor)
            
#             p_i = p_i / len(os.listdir(class_i)) 

#             dataset1_vectorlist.append(p_i)
            
#         # 遍历第二个数据集的标签
#         dataset2_vectorlist = []
#         for j in tqdm(label_class2):
#             ##* 对于 dataset2 的 第j类 图像，提取出平均特征 p_j
#             class_j = os.path.join(dataset2, j)
            
#             p_j = 0
#             # 遍历类别下的每张图片
#             for image_name in os.listdir(class_j):
#                 image_path = os.path.join(class_j, image_name)
#                 image_tensor = preprocess_image(image_path)
#                 image_tensor = image_tensor.to(device)
#                 p_j += model.encode_image(image_tensor)
            
#             p_j = p_j / len(os.listdir(class_j))    
#             dataset2_vectorlist.append(p_j)        
            
            
#         # 创建 权重矩阵
#         weight_matrix = np.ones((len(label_class1),len(label_class2)))  
#         weight_matrix = weight_matrix/np.sum(weight_matrix)
         
#         # 计算
#         EMD = 0
#         for i in range(len(dataset1_vectorlist)):
#             for j in range(len(dataset2_vectorlist)):
                
#                 ##* 将两个求二范数  d_ij
#                 EMD += weight_matrix[i][j] * torch.norm(dataset1_vectorlist[i] - dataset2_vectorlist[j], p=2)

#     return math.e ** (-1*alpha*EMD)    

## Compute

In [27]:
# # CUB
# CUB_path = 'data/CUB_200_2011/images/'

# # Stanford Cars
# Cars_path = 'data/stanford_cars/images/'

# # EuroSAT
# EuroSAT_path = 'data/EuroSAT_RGB/'

# # Plant disease
# Plant_path = 'data/Plant_disease/images'

# # mini-imagenet
# mini_path = 'data/mini-imagenet/images/'

Note the hierarchy: `data_dir`->`(class1, class2, ...)`->`(image1.jpg, image2.jpg)`

In [ ]:
similarity_withSource(CUB_path)
similarity_withSource(Cars_path)
similarity_withSource(EuroSAT_path)
similarity_withSource(Plant_path)